## Preprocessing

In [235]:
# Import our dependencies
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [236]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
dropped_df = application_df.drop(columns=['EIN', 'NAME'])

# Display the updated DataFrame to verify the changes
print(dropped_df)

      APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0                  T10       Independent          C1000    ProductDev   
1                   T3       Independent          C2000  Preservation   
2                   T5  CompanySponsored          C3000    ProductDev   
3                   T3  CompanySponsored          C2000  Preservation   
4                   T3       Independent          C1000     Heathcare   
...                ...               ...            ...           ...   
34294               T4       Independent          C1000    ProductDev   
34295               T4  CompanySponsored          C3000    ProductDev   
34296               T3  CompanySponsored          C2000  Preservation   
34297               T5       Independent          C3000    ProductDev   
34298               T3       Independent          C1000  Preservation   

       ORGANIZATION  STATUS     INCOME_AMT SPECIAL_CONSIDERATIONS   ASK_AMT  \
0       Association       1              0  

In [237]:
# Determine the number of unique values in each column.
unique_values = dropped_df.nunique()

# Display the number of unique values for each column
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [238]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
# Step 1: Get the value counts for APPLICATION_TYPE
application_type_counts = dropped_df['APPLICATION_TYPE'].value_counts()

# Step 2: Define a threshold for infrequent application types
threshold = 10

# Step 3: Replace infrequent application types with 'Other'
dropped_df['APPLICATION_TYPE'] = dropped_df['APPLICATION_TYPE'].apply(lambda x: 'Other' if application_type_counts[x] < threshold else x)

# Display the updated DataFrame to verify the changes
print(dropped_df['APPLICATION_TYPE'].value_counts())


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: count, dtype: int64


In [239]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_type_counts[application_type_counts < threshold].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: count, dtype: int64

In [240]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = dropped_df['CLASSIFICATION'].value_counts()

In [241]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_filtered = classification_counts[classification_counts > 1]

In [242]:
# Choose a cutoff value and create a list of classifications to be replaced
cutoff_value = 10

# use the variable name `classifications_to_replace`
classifications_to_replace = classification_counts_filtered[classification_counts_filtered < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
Other       72
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C2561        1
C1580        1
C1900        1
C1820        1
C6100        1
C1248        1
C2600        1
C1370        1
C5200        1
C4200        1
C2190        1
C1236        1
C2570        1
C2500        1
C3700        1
C1570        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120        1
C8210        1
C4500        1
C1245        1
C2150        1
Name: count, dtype: int64

In [243]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = ['APPLICATION_TYPE', 'CLASSIFICATION', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'STATUS', 'SPECIAL_CONSIDERATIONS']

# Convert categorical columns to numeric using pd.get_dummies
categorical_df = pd.get_dummies(dropped_df, columns=categorical_columns)

# Step 3: Display the updated DataFrame to verify the changes
print(categorical_df.head())

      INCOME_AMT  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0              0     5000              1                   False   
1         1-9999   108590              1                   False   
2              0     5000              0                   False   
3    10000-24999     6692              1                   False   
4  100000-499999   142590              1                   False   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T12  APPLICATION_TYPE_T13  \
0                  True                 False                 False   
1                 False                 False                 False   
2                 False                 False                 False   
3                 False                 False                 False   
4                 False                 False                 False   

   APPLICATION_TYPE_T19  APPLICATION_TYPE_T2  APPLICATION_TYPE_T3  ...  \
0                 False                False                False  ...   
1               

In [244]:
# Split our preprocessed data into our features and target arrays
# Assuming 'IS_SUCCESSFUL' is the target column and application_df is your preprocessed DataFrame
X = categorical_df.drop(columns=['IS_SUCCESSFUL'])  # Features (all columns except the target)
y = categorical_df['IS_SUCCESSFUL']  # Target column

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify the split by checking the shapes of the resulting datasets
print("Training Features Shape:", X_train.shape)
print("Testing Features Shape:", X_test.shape)
print("Training Target Shape:", y_train.shape)
print("Testing Target Shape:", y_test.shape)

Training Features Shape: (27439, 105)
Testing Features Shape: (6860, 105)
Training Target Shape: (27439,)
Testing Target Shape: (6860,)


In [245]:
from sklearn.model_selection import train_test_split

# Convert categorical variables to numeric
application_df = pd.get_dummies(application_df)

# Convert Range Strings to Numeric Values
def convert_range_to_numeric(value):
    if isinstance(value, str):
        # Handle cases with 'M' for million
        if 'M' in value:
            value = value.replace('M', '')
            if '-' in value:
                low, high = value.split('-')
                return (float(low) + float(high)) / 2 * 1e6
            elif '+' in value:
                value = value.replace('+', '')
                return float(value) * 1e6  # Treat '50+' as just 50M
            else:
                return float(value) * 1e6
        
        # Handle cases with 'K' for thousand
        elif 'K' in value:
            value = value.replace('K', '')
            if '-' in value:
                low, high = value.split('-')
                return (float(low) + float(high)) / 2 * 1e3
            elif '+' in value:
                value = value.replace('+', '')
                return float(value) * 1e3  # Treat '50+' as just 50K
            else:
                return float(value) * 1e3
        
        # Handle cases with '-' but no 'M' or 'K'
        elif '-' in value:
            low, high = value.split('-')
            return (float(low) + float(high)) / 2
        
        # Handle cases with '+' without 'M' or 'K'
        elif '+' in value:
            value = value.replace('+', '')
            return float(value)  # Treat '50+' as just 50

    # If not a string or does not match the expected formats, return the original value
    return value

# Ensure that the column exists and apply this function to the relevant columns
if 'INCOME_AMT' in application_df.columns:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].apply(convert_range_to_numeric)

# Split the data into features (X) and target (y)
X = application_df.drop(columns=['IS_SUCCESSFUL'])  # Features (all columns except the target)
y = application_df['IS_SUCCESSFUL']  # Target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler on the training data and scale the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

MemoryError: Unable to allocate 512. MiB for an array with shape (19568, 27439) and data type bool

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

c:\Users\lexis\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8)              │       157,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            45 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 157,387 (614.79 KB)

 Trainable params: 157,387 (614.79 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Drop additional columns that may not be important
X_optimized1 = X.drop(columns=['USE_CASE', 'AFFILIATION'])  # Replace with actual column names

# Re-split and scale the data
X_train_opt1, X_test_opt1, y_train_opt1, y_test_opt1 = train_test_split(X_optimized1, y, test_size=0.2, random_state=42)
X_train_opt1_scaled = scaler.fit_transform(X_train_opt1)
X_test_opt1_scaled = scaler.transform(X_test_opt1)

# Define a new model with more neurons in the first hidden layer
model_opt1 = Sequential()
model_opt1.add(Dense(units=16, input_dim=X_train_opt1_scaled.shape[1], activation='relu'))  # Increased neurons
model_opt1.add(Dense(units=8, activation='relu'))
model_opt1.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model_opt1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_opt1.fit(X_train_opt1_scaled, y_train_opt1, epochs=10, shuffle=True, verbose=2)

MemoryError: Unable to allocate 6.49 GiB for an array with shape (435539247, 2) and data type int64

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5578 - acc: 0.7263
Loss: 0.557812534073699, Accuracy: 0.7262973785400391


In [ ]:
# Export our model to HDF5 file
nn.save("Module_21_Challenge_model_LS.h5")